In [604]:
#まずは今回使うrequestsとBeautifulSoupをインポートします。
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [605]:
REQUEST_URL = "https://suumo.jp/jj/chintai/ichiran/FR301FC005/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&po2=99&pc=100"
#東京都千代田区の賃貸住宅[賃貸マンション・アパート]情報 検索結果<部屋ごとに表示> 表示件数100件

html = requests.get(REQUEST_URL)
soup = BeautifulSoup(html.content, "html.parser")


In [606]:
#検索結果のページ数を取得する（ページ数でリンクを取得） １ページ、２〜５ページ、６ページ以上で場合分けが必要？
# ２ページ以上だと同じで大丈夫？

next_page = soup.find('p', class_="pagination-parts") #次へ　の部分を抽出一つだけのようなのでこれで。。。
try:
  next_page = next_page.find_previous('li') #'次へ'の前のliを抽出
  next_page = next_page.text #text部分を抽出
  pages = int(next_page)
except:
  pages = 1

pages


152

In [609]:
#多すぎるので便宜的にpagesを10とする

pages = 10


In [619]:
property_info_list = []  # 情報を入れるリストを作成

for page in range(1, pages + 1):  #検索結果のページごとにデータを収集する
    request_url2 = REQUEST_URL + "&page=" + str(page) #検索結果のページの後に追加することで別のページになる

    htmls = requests.get(request_url2)
    soup2 = BeautifulSoup(htmls.content, "html.parser")

    detail_links = soup2.find_all('a', class_="js-cassetLinkHref") #各物件の詳細情報の記載ページのリンクを取得する

    detail_urls = []

    for href_tag in detail_links:
        link = href_tag.get('href')
        link = 'https://suumo.jp' + link  # 欠けているので追加
        detail_urls.append(link)

    # 詳細情報（住所、間取り、階数）を取得する
    detail_data = []

    for detail_url in detail_urls:
        url = requests.get(detail_url)
        soup3 = BeautifulSoup(url.content, "html.parser")

        # やっぱりこのページから物件名も取得することにしました
        property_element = soup3.select_one('h1.section_h1-header-title')  # h1
        property_name = None

        if property_element:
            property_text = property_element.text.strip()
            pos1 = property_text.find(' - ')  # 区切りのハイフンなどを目印にして抽出
            property_name = property_text[:pos1]

        # 家賃についても同様に。
        rent_number = None

        rent = soup3.find_all('div', class_="property_view_main-emphasis")  # 全く絞れないので、万円を含んでいるものだけを抽出する
        for div_tag in rent:
            rent_number = div_tag.text.strip()
            if "万円" in rent_number:
                rent_number = rent_number.replace('万円', '')  # 万円を削除
                rent_number = int(float(rent_number) * 10000)  # 単位を円にする。小数点がついているのでfloat、からのint

        # 物件の詳細ページから階数を取得 下にある表から取得
        floor_number = None
        try:
          floor_tags = soup3.find_all('th', class_="data_01", string="階建")  # thのdata_01クラス,stringで指定

          if floor_tags:
              for th_tag in floor_tags:
                  next_td = th_tag.find_next('td')
                  if next_td:
                      floor_number_text = next_td.text.strip()
                      pos = floor_number_text.find('階/')
                      floor_number_text = floor_number_text[:pos]
                      floor_number = int(floor_number_text)  # int型に変える
        except:     # 戸建ての賃貸で複数階の物件でエラーが出たので、intにしないでそのまま入れることに
          floor_tags = soup3.find_all('th', class_="data_01", string="階建")  # thのdata_01クラス,stringで指定

          if floor_tags:
              for th_tag in floor_tags:
                  next_td = th_tag.find_next('td')
                  if next_td:
                      floor_number_text = next_td.text.strip()
                      pos = floor_number_text.find('階/')
                      floor_number_text = floor_number_text[:pos]
          pass

        # 住所を取得
        location = None
        location = soup3.find_all('span', class_="property_view_detail-header-title")

        for div_tag in location:
            if div_tag.text.strip() == "所在地":  # classだけでは絞れなかった
                next_div = div_tag.find_next('div')
                if next_div:
                    location = next_div.text.strip()
                    break

        # 間取り
        floor_plan = None
        floor_plan = soup3.find_all('div', class_="property_data-title")

        for div_tag2 in floor_plan:
            if div_tag2.text.strip() == "間取り":  # classだけでは絞れなかった
                next_div2 = div_tag2.find_next('div')
                if next_div2:
                    floor_plan = next_div2.text.strip()
                    break

        detail_info = {
            "物件名": property_name,
            "家賃": rent_number,
            "住所": location,
            "階": floor_number,
            "間取り": floor_plan,
        }

        property_info_list.append(detail_info)

df_property_info = pd.DataFrame(property_info_list)

df_property_info


,物件名,家賃,住所,階,間取り
0,ブライズ神田岩本町 1502号室,224000.0,東京都千代田区岩本町２,15.0,2LDK
1,ＫＷレジデンス麹町,532000.0,東京都千代田区麹町３,10.0,2LDK
2,ザ・パークハウス麹町テラス 701号室,350000.0,東京都千代田区麹町３,7.0,2LDK
3,ザ・パークハウス麹町テラス 701号室,350000.0,東京都千代田区麹町３,7.0,2LDK
4,ザ・パークハウス麹町テラス 701号室,350000.0,東京都千代田区麹町３,7.0,2LDK
...,...,...,...,...,...
995,ＫＷレジデンス麹町,162000.0,東京都千代田区麹町３,6.0,1K
996,グランクオール秋葉原,90000.0,東京都千代田区神田佐久間町３,3.0,1K
997,ＪＲ山手線 秋葉原駅 4階建 築3年,90000.0,東京都千代田区神田佐久間町３,3.0,1K
998,ＪＲ山手線 秋葉原駅 3階建 築3年,90000.0,東京都千代田区神田佐久間町３,3.0,1K


In [616]:
df_property_info.to_csv("suumo_sample.csv")


以下検討の残骸

In [519]:


request_url2 = "https://suumo.jp/jj/chintai/ichiran/FR301FC005/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&po2=99&pc=100"

htmls = requests.get(request_url2)
soup2 = BeautifulSoup(htmls.content, "html.parser")


In [ ]:
#物件名と詳細情報リンクの表示されている部分を抽出
property = soup2.find_all('a', class_="js-cassetLinkHref")

#物件名を取得。文字がリンクに使われているのでそのテキストを取得し格納

property_texts = []

for a_tag in property:
    property_texts.append(a_tag.text.strip())

property_texts


In [ ]:
#’詳細を見る’のURLを取得する

property = soup2.find_all('a', class_="js-cassetLinkHref")

property


In [596]:
detail_urls = []

for href_tag in property:
    link = href_tag.get('href')
    link = 'https://suumo.jp'+ link #欠けているので追加
    detail_urls.append(link)

detail_urls


['https://suumo.jp/chintai/bc_100355778748/',
 'https://suumo.jp/chintai/bc_100355773975/',
 'https://suumo.jp/chintai/bc_100355909050/',
 'https://suumo.jp/chintai/bc_100355923442/',
 'https://suumo.jp/chintai/bc_100355909048/',
 'https://suumo.jp/chintai/bc_100355923443/',
 'https://suumo.jp/chintai/bc_100355923727/',
 'https://suumo.jp/chintai/bc_100355734599/',
 'https://suumo.jp/chintai/bc_100355769509/',
 'https://suumo.jp/chintai/bc_100355772583/',
 'https://suumo.jp/chintai/bc_100355772578/',
 'https://suumo.jp/chintai/bc_100355769027/',
 'https://suumo.jp/chintai/bc_100338052690/',
 'https://suumo.jp/chintai/bc_100341277227/',
 'https://suumo.jp/chintai/bc_100355819183/',
 'https://suumo.jp/chintai/bc_100355774023/',
 'https://suumo.jp/chintai/bc_100355767881/',
 'https://suumo.jp/chintai/bc_100355811612/',
 'https://suumo.jp/chintai/bc_100355807708/',
 'https://suumo.jp/chintai/bc_100355886793/',
 'https://suumo.jp/chintai/bc_100355767866/',
 'https://suumo.jp/chintai/bc_1003

In [587]:
property = soup2.find_all('a', class_="js-cassetLinkHref")


In [586]:
detail_urls


['https://suumo.jp/chintai/jnc_000080299630/?bc=100346871399']

In [ ]:
#家賃を取得する

rents = soup2.find_all('div', class_="detailbox-property-point") #classの指定で取得

rent_amounts = []

for div_tag in rents:
    rent = div_tag.text.strip() #テキストだけ取得する
    rent = rent.replace('万円', '') #万円を削除
    rent = int(float(rent)*10000) #str、.があるのでfloatにしてからintに
    rent_amounts.append(rent)

rent_amounts


In [ ]:
#管理費を取得する。classがないので家賃の次、という条件で取得（これでいいのか？）
management_fees = []

for div_tag2 in rents:
  management_fee = div_tag2.find_next('div').text.strip()
  if management_fee == "-": #０円の時は-になってる・・
    management_fees.append(0) #ここがなぜかうまくいかない。ハイフンが認識されない
  else:
    management_fee = management_fee.replace("管理費 ", "").replace("円", "") #管理費 20000円のようになるので不要分を削除
    try:
      management_fee = int(management_fee)
      management_fees.append(management_fee)
    except:
      management_fees.append(0) #うまくいかないため、数値でない場合は０に

management_fees


物件名
-住所
-間取り
家賃
-階数

In [ ]:
#やっぱり物件名も家賃も全て詳細のページから取得する。物件名はh1

property =


In [535]:
#階の情報が詳細を見る野崎のページにしかないのでそこから取得する(住所、間取りも)


REQUEST_URL2 = "https://suumo.jp/chintai/bc_100355401564/"
#パークハウス三番町 - アエラス新橋店 (株)アエラス.ERが提供する賃貸物件情報


url = requests.get(REQUEST_URL2)
soup3 = BeautifulSoup(url.content, "html.parser")


In [545]:
#やっぱり物件名も家賃も全て詳細のページから取得する。物件名はh1

property = soup3.select_one('h1', class_="section_h1-header-title") #h1

for h1_tag in property:
    if property:
      property_text = property.text.strip()

pos1 = property_text.find(' - ')  #区切のハイフンなどを目印にして抽出
property_name = property_text[:pos1]
property_name


'パークハウス三番町'

In [554]:
rent = soup3.find_all('div', class_="property_view_main-emphasis") #全く絞れないので、万円を含んでいるものだけを抽出する

for div_tag in rent:
  rent_number = div_tag.text.strip()
  if "万円" in rent_number:
    rent_number = rent_number.replace('万円', '') #万円を削除
    rent_number = int(float(rent_number)*10000) #単位を円にする。小数点がついているのでfloat、からのint

rent_number


630000

In [617]:
#物件の詳細ページから階数を取得 下にある表から取得
floor = soup3.find_all('th', class_="data_01") #thのdata_01クラスで指定

for th_tag in floor:
    if th_tag.text.strip() == "階建": #classだけでは絞れなかったので文字が一致しているもので絞り込み
        next_td = th_tag.find_next('td')
        if next_td:
            print(next_td.text.strip())
            break

#階以降を削除する

floor_number = next_td.text.strip()
pos2 = floor_number.find('階/') #階/までの文字数を取得
floor_number = floor_number[:pos2]
type(floor_number)    #型がstrの模様
floor_number = int(floor_number) #int型に変える

floor_number


7階/8階建


7

In [517]:
#住所を取得

location = soup3.find_all('span', class_="property_view_detail-header-title")

for div_tag in location:
  if div_tag.text.strip() == "所在地":  #classだけでは絞れなかった
    next_div = div_tag.find_next('div')
    if next_div:
      location = next_div.text.strip()
      break

location


'東京都千代田区三番町'

In [518]:
#間取り

floor_plan = soup3.find_all('div', class_="property_data-title")

for div_tag2 in floor_plan:
  if div_tag2.text.strip() == "間取り":  #classだけでは絞れなかった
    next_div2 = div_tag2.find_next('div')
    if next_div2:
      floor_plan = next_div2.text.strip()
      break

floor_plan   #全て取得完了


'3LDK'

In [ ]:
<div class="property_data-title">間取り</div>


In [ ]:
key_money = soup.find('span', class_="property_view_note-emphasis").find_next('span').find_next('span').find_next('span').get_text(strip = True)


In [ ]:
<span class="property_view_detail-header-title">所在地</span>


In [378]:
#住所を取得する。（これも4つ先のtdで取得）

locations = []

for div_tag3 in rents:
  management_fee = div_tag3.find_next('td').text.strip()
  locations.append(location)


In [102]:
# Webページを取得して解析する
REQUEST_URL = "https://suumo.jp/chintai/jnc_000086641107/?bc=100355089468"

html = requests.get(REQUEST_URL)
soup = BeautifulSoup(html.content, "html.parser")


In [179]:
property_name = soup.find('title').text.strip()

print(property_name)


【SUUMO】ラセーヌポンヌフ／東京都足立区入谷５／舎人駅の賃貸・部屋探し情報（000086641107） | 賃貸マンション・賃貸アパート


In [181]:
# 【SUUMO】 ／で囲まれた文字列を抽出する。他にも／はあるけど大丈夫かな・・・
import re

s = property_name
pattern = r'\【SUUMO】(.+?)\／'  # []で囲まれた文字列を抽出する正規表現
match = re.search(pattern, s)
if match:
    captured = match.group(1)
    print(captured)


ラセーヌポンヌフ


In [103]:
fee = (soup.find('span', class_="property_view_note-emphasis").find_next('span')).get_text(strip = True) #家賃の隣にある管理費・共益費を取得

print(fee)


管理費・共益費: 5500円


In [104]:
replaced_fee = deposit.replace('管理費・共益費:', '').replace('\xa0', '').replace('円', '') #不要な文字を削除
replaced_fee = int(replaced_deposit) #int型に変換
print(type(replaced_fee))


<class 'int'>


In [182]:
#敷金のデータを取得する
sec_dep = (soup.find('span', class_="property_view_note-emphasis").find_next('span').find_next('span')).get_text(strip = True) #家賃の次の次

sec_dep


'敷金:\xa0-'

In [183]:
replaced_sec_dep = sec_dep.replace('敷金:', '').replace('\xa0', '').replace('.', '').replace('万円', '') #「.」も文字型なので削除
replaced_sec_dep = (int(replaced_sec_dep))*1000 #int型にして1000をかけて数値に変換
replaced_sec_dep


ValueError: invalid literal for int() with base 10: '-'

In [107]:
#礼金も同様に取得
key_money = soup.find('span', class_="property_view_note-emphasis").find_next('span').find_next('span').find_next('span').get_text(strip = True)

key_money


'礼金:\xa08.9万円'

In [127]:
replaced_key_money = key_money.replace('礼金:', '').replace('\xa0', '')
replaced_key_money


'8.9万円'

In [129]:
if replaced_key_money == '-':
  replaced_key_money=replaced_key_money.replace('-', '0')
  replaced_key_money= int(replaced_key_money)
else:
  replaced_key_money = str(replaced_key_money).replace('.', '').replace('万円', '')
  replaced_key_money = (int(replaced_key_money))*1000
  print(replaced_key_money)


89000


In [130]:
#保証金も同様に

deposit = soup.find('span', class_="property_view_note-emphasis").find_next('span').find_next('span').find_next('span').find_next('span').get_text(strip = True)

deposit = deposit.replace('保証金:', '').replace('\xa0', '') #不要な文字を削除

#0円の時は-なのでそれを変換する。あるいは単位を万円から縁に変換するよう計算
if deposit == '-':
  deposit=deposit.replace('-', '0')
  deposit = int(deposit)
  print(deposit)
else:
  deposit = str(deposit).replace('.', '').replace('万円', '')
  deposit = (int(deposit))*1000
  print(deposit)


0


In [133]:
#敷引・償却

depre = soup.find('span', class_="property_view_note-emphasis").find_next('span').find_next('span').find_next('span').find_next('span').find_next('span').get_text(strip = True)

depre = depre.replace('敷引・償却:', '').replace('\xa0', '') #不要な文字を削除

#0円の時は-なのでそれを変換する。あるいは単位を万円から縁に変換するよう計算
if depre == '-':
  depre = depre.replace('-', '0')
  depre = int(deposit)
  print(depre)
else:
  depre = str(depre).replace('.', '').replace('万円', '')
  depre = (int(depre))*1000
  print(depre)


0


In [135]:
#下の表から住所を取得

location = (soup.find('th', string='所在地').find_next('td')).text.strip()

location


'東京都足立区入谷５'

In [136]:
#下の表から間取りを取得
floor_plan = (soup.find('th', string='間取り').find_next('td')).text.strip()

floor_plan


'1LDK'

In [138]:
#下の表から階を取得
floor= (soup.find('th', string='階').find_next('td')).text.strip().replace('階', '')

floor


'5'

In [53]:
#リンク先のURLを取得する

properties = soup.find('', ='', class = '')
a_tag = properties.find_all('a')
print([x['heref'] for x in a_tag])


SyntaxError: invalid syntax (3647102302.py, line 1)

In [54]:
物件名
-住所
-間取り
-家賃
-階数


NameError: name '物件名' is not defined

検索結果から詳細を見る　のリンクを抽出する

In [225]:
RESULT_URL = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13121&ta=13&cb=0.0&ct=8.0&co=1&co=3&co=4&ts=1&ts=3&et=7&mb=0&mt=9999999&cn=5&tc=0400301&tc=0400501&fw2="

page = requests.get(RESULT_URL)
soup2 = BeautifulSoup(page.content, "html.parser")


In [ ]:
result = soup2.find_all('a', class_="js-cassette_link_href cassetteitem_other-linktext" )
result


In [243]:
len(result)


57

In [247]:
property_links = []
for i in range(len(result)):
  try:
    link = result[i].get("href")
    link = 'https://suumo.jp'+link
    property_links.append(link)
  except:
    pass


In [248]:
property_links


['https://suumo.jp/chintai/jnc_000058305180/?bc=100354154893',
 'https://suumo.jp/chintai/jnc_000068023052/?bc=100354682823',
 'https://suumo.jp/chintai/jnc_000084251523/?bc=100352100611',
 'https://suumo.jp/chintai/jnc_000086543359/?bc=100355408502',
 'https://suumo.jp/chintai/jnc_000071408422/?bc=100352689248',
 'https://suumo.jp/chintai/jnc_000080292179/?bc=100320294815',
 'https://suumo.jp/chintai/jnc_000080292180/?bc=100354549975',
 'https://suumo.jp/chintai/jnc_000080292189/?bc=100315942525',
 'https://suumo.jp/chintai/jnc_000086380170/?bc=100346561786',
 'https://suumo.jp/chintai/jnc_000086543308/?bc=100354600084',
 'https://suumo.jp/chintai/jnc_000086635132/?bc=100349884938',
 'https://suumo.jp/chintai/jnc_000086416638/?bc=100355462152',
 'https://suumo.jp/chintai/jnc_000086472475/?bc=100354786630',
 'https://suumo.jp/chintai/jnc_000086399807/?bc=100346561692',
 'https://suumo.jp/chintai/jnc_000086590453/?bc=100354651440',
 'https://suumo.jp/chintai/jnc_000086548445/?bc=1003464

In [ ]:
#結果のページ数を取得
